In [6]:
from helpers import *

import gt4py.next as gtx

In [7]:
def diffusion_step_numpy(
    e2c2v: np.array,
    v2e: np.array,
    TE: np.array,
    TE_t: np.array,
    inv_primal_edge_length: np.array,
    inv_vert_vert_length: np.array,
    nnbhV: np.array,
    boundary_edge: np.array,
    kappa: float,
    dt: float,
) -> np.array:

    # initialize
    TEinit = TE

    # predict
    TE = TEinit + 0.5*dt*TE_t

    # interpolate temperature from edges to vertices
    TV = neighbor_sum(TE(v2e), axis=1) / nnbhV

    # compute nabla2 using the finite differences
    TEnabla2 = neighbor_sum(
        (TV(e2c2v[0]) + TV(e2c2v[1])) * inv_primal_edge_length ** 2
        + (TV(e2c2v[3]) + TV(e2c2v[4])) * inv_vert_vert_length ** 2,
        axis=1,
    )

    TEnabla2 = TEnabla2 - (
        (2.0 * TE * inv_primal_edge_length ** 2)
        + (2.0 * TE * inv_vert_vert_length ** 2)
    )

    # build ODEs
    TE_t = where(
        boundary_edge,
        0.,
        kappa*TEnabla2,
    )

    # correct
    TE = TEinit + dt*TE_t

In [8]:
@gtx.field_operator
def diffusion_step(
    TE: gtx.Field[Dims[E], float],
    TE_t: gtx.Field[Dims[E], float],
    inv_primal_edge_length: gtx.Field[Dims[E], float],
    inv_vert_vert_length: gtx.Field[Dims[E], float],
    nnbhV: gtx.Field[Dims[V], float],
    boundary_edge: gtx.Field[Dims[E], bool],
    kappa: float,
    dt: float,
) -> gtx.tuple[
    gtx.Field[Dims[E], float],
    gtx.Field[Dims[E], float],
]:

    # initialize
    TEinit = TE

    # predict
    TE = TEinit + 0.5*dt*TE_t

    # interpolate temperature from edges to vertices
    TV = neighbor_sum(TE(V2E), axis=V2EDim) / nnbhV

    # compute nabla2 using the finite differences
    TEnabla2 = neighbor_sum(
        (TV(E2C2V[0]) + TV(E2C2V[1])) * inv_primal_edge_length ** 2
        + (TV(E2C2V[3]) + TV(E2C2V[4])) * inv_vert_vert_length ** 2,
        axis=E2C2VDim,
    )

    TEnabla2 = TEnabla2 - (
        (2.0 * TE * inv_primal_edge_length ** 2)
        + (2.0 * TE * inv_vert_vert_length ** 2)
    )

    # build ODEs
    TE_t = where(
        boundary_edge,
        0.,
        kappa*TEnabla2,
    )

    # correct
    TE = TEinit + dt*TE_t
    
    return TE_t, TE

DSLError: Undeclared symbol 'E2C2V'.
  File "/private/var/folders/2b/_2y31vzs4sl_7rngh2yghbpw0000gn/T/ipykernel_56953/2481047861.py", line 27

In [9]:
def test_diffusion_step():
    u = random_field((n_edges), E)
    v = random_field((n_edges), E)
    nx = random_field((n_edges), E)
    ny = random_field((n_edges), E)
    L = random_field((n_edges), E)
    dualL = random_field((n_edges), E)
    A = random_field((n_cells), C)
    dualA = random_field((n_vertices), V)
    edge_orientation_vertex = random_field((n_cells, 6), V, V2EDim)
    edge_orientation_cell = random_field((n_cells, 3), C, C2EDim)

    laplacian_ref = laplacian_numpy(
        c2e_table,
        u.asnumpy(),
        v.asnumpy(),
        nx.asnumpy(),
        ny.asnumpy(),
        L.asnumpy(),
        A.asnumpy(),
        edge_orientation.asnumpy(),
    )

    c2e_connectivity = gtx.NeighborTableOffsetProvider(c2e_table, C, E, 3)

    laplacian_gt4py = zero_field((n_edges), E)

    laplacian_fvm(
        u, v, nx, ny, L, A, edge_orientation, out = divergence_gt4py, offset_provider = {C2E.value: c2e_connectivity}
    )
    
    assert np.allclose(divergence_gt4py.asnumpy(), divergence_ref)

In [10]:
test_diffusion_step()
print("Test successful")

AttributeError: 'int' object has no attribute 'shape'